In [1]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from jaxtyping import Float, Int
from typing import List, Optional, Tuple, Dict
import sys
from functools import partial
from tqdm import tqdm
import json
import seaborn as sns
import pandas as pd
import multiprocessing
import pickle

from request_patching import request_patch_one_pair, create_patch_request_dict, baseline_completion, baseline_completion_plus
from models import get_model_from_name

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device =', device)
torch.set_grad_enabled(False)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

/home/jsalle/.conda/envs/athena_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = cpu
CUDA is not available. Listing CPUs instead.
48


In [2]:
dict = create_patch_request_dict('gpt2-small',
                                 dataset='dialogs_2')

Number of stories: 3


In [3]:
dict = create_patch_request_dict('pythia-1b',
                                 dataset='dialogs_2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 3


In [4]:
dict = create_patch_request_dict('pythia-2.8b',
                                 dataset='dialogs_2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of stories: 3


In [31]:
from request_patching import get_first_token_from_str

model, tokenizer = get_model_from_name('pythia-2.8b')
token_ids = tokenizer.encode("I feel jealous")
for token_id in reversed(token_ids):
        str_token = tokenizer.decode(token_id)
        print(str_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 jealous
 feel
I


In [13]:
dict

{'pair_0': {'context_1': "Alice: Hey, Bob! It's so nice to see you. How was your day?\nBob: Hi, Alice. It's good to see you too. My day was... okay, I guess. A bit on the nerve-wracking side. How about you? You look pretty cheerful today.\nAlice: Oh, I had a fantastic day! The weather was perfect for my morning jog, and then I spent some time in the park. It was so relaxing and just what I needed. But what happened with you? Why was your day nerve-wracking?\nBob: Well, I had to give a presentation at work today. Even though I prepared a lot, I couldn't shake off the fear of messing it up. Standing in front of everyone, I felt like my mind went blank for a moment. It was terrifying.\nAlice: I can imagine that was really scary, but I'm sure you did better than you think. You're always so well-prepared. Maybe it wasn't as bad as it felt in the moment?\nBob: Maybe, but it's hard to tell. I just keep replaying it in my head and wondering what everyone thought. It's difficult to shake off th